In [1]:
"""
import os
import torch
from huggingface_hub import login
"""

'\nimport os\nimport torch\nfrom huggingface_hub import login\n'

In [2]:
"""
# Setting working directory
work_dir = "/storage/work/~~~" 
os.environ['HF_HOME'] = f"{work_dir}/.cache/huggingface"
os.environ['TRANSFORMERS_CACHE'] = f"{work_dir}/.cache/huggingface"

# Huggingface token
hf_token = "hf_~~~" 
login(token = hf_token)
"""

'\n# Setting working directory\nwork_dir = "/storage/work/~~~" \nos.environ[\'HF_HOME\'] = f"{work_dir}/.cache/huggingface"\nos.environ[\'TRANSFORMERS_CACHE\'] = f"{work_dir}/.cache/huggingface"\n\n# Huggingface token\nhf_token = "hf_~~~" \nlogin(token = hf_token)\n'

In [3]:
%pip install llama-index-embeddings-ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
# from llama_index.llms.huggingface import HuggingFaceLLM
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

In [5]:
"""
Settings.embed_model = HuggingFaceEmbedding(model_name = "BAAI/bge-small-en-v1.5")

Settings.llm = HuggingFaceLLM(
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_name = "meta-llama/Meta-Llama-3-8B-Instruct",
    context_window = 8192,
    max_new_tokens = 256,
    generate_kwargs = {"temperature": 0.1, "do_sample": False},
    device_map = "auto",
    model_kwargs = {"torch_dtype": torch.float16, "load_in_8bit": False} 
)
"""

Settings.embed_model = OllamaEmbedding(model_name = "llama3")

Settings.llm = Ollama(
    model = "llama3",
    request_timeout = 360.0,
    temperature = 0.1
)

In [6]:
# Loading documents
reader = SimpleDirectoryReader(
    input_dir=".", 
    required_exts=[".pdf", ".docx"] 
)
documents = reader.load_data()

# Creating index and engine
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

2026-01-28 01:41:25,625 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2026-01-28 01:41:26,185 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2026-01-28 01:41:28,662 - INFO - HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


In [7]:
question = "What is difference between prompt sensitivity and model sensitivity?"

In [ ]:
# Input (question)
print(f"Question: {question}\n")
response = query_engine.query(question)

## Retrieved snippets
print("-" * 50)
for i, node in enumerate(response.source_nodes, 1):
    # chunk ID
    page = node.metadata.get('page_label', 'N/A')
    filename = node.metadata.get('file_name', 'N/A')
    score = node.score if node.score else 0.0
    
    # snippet text
    content_preview = node.text[:200].replace('\n', ' ') + "..."
    
    print(f"Chunk #{i} | Score: {score:.4f} | Source: {filename} (Page {page})")
    print(f"Content: \"{content_preview}\"")
    print("-" * 50)

# Output (answer)
print("RESPONSE:")
print(response)
print("-" * 50)

# Source
print("Citations:")
for node in response.source_nodes:
    print(f"- Page {node.metadata['page_label']}: {node.text[:100]}...")

2026-01-28 01:41:28,867 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"


Question: What is difference between prompt sensitivity and model sensitivity?



2026-01-28 01:41:40,550 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


--------------------------------------------------
Chunk #1 | Score: 0.6559 | Source: LLM_SSRI.pdf (Page 4)
Content: "4/13 Why We Need a Systematic Framework Can an LLM do this task? V.S. How can we validate and document LLM use so that our inferences remain credible?..."
--------------------------------------------------
Chunk #2 | Score: 0.5936 | Source: LLM_SSRI.pdf (Page 8)
Content: "8/13 Step 3-Stability Evaluation:Reduce cross-prompt/model variance Prompt Sensitivity - Test multiple prompt templates / phrasings - Assess variation in metrics across prompts Model sensitivity - Com..."
--------------------------------------------------
RESPONSE:
Prompt Sensitivity refers to testing multiple prompt templates/phrasings and assessing variation in metrics across prompts. This involves evaluating how different prompts affect the performance of a language model.

Model Sensitivity, on the other hand, involves comparing models (e.g., GPT, Llama, Deepseek) and examining where they disagree 

In [15]:
# Without RAG
raw_response = Settings.llm.complete("Who is Cassandra Tai?")
print(f"[Without RAG]:\n{raw_response}\n")

# Using RAG
rag_response = query_engine.query("Who is Cassandra Tai?")
print(f"[Using RAG]:\n{rag_response}")

2026-01-28 01:45:23,735 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


[Without RAG]:
Cassandra Tai is a popular Australian YouTuber and beauty influencer. She was born on August 24, 1994, in Melbourne, Australia. Tai initially gained fame on YouTube by sharing her makeup tutorials, product reviews, and lifestyle vlogs.

Over time, she expanded her content to include skincare routines, hair care tips, and even cooking recipes. Her relatable personality, honesty, and authenticity have made her a beloved figure among her fans, who affectionately call themselves "Taisies."

Cassandra Tai has collaborated with various beauty brands and has been featured in several publications, including Australian Vogue and Harper's Bazaar. She has also spoken at events like the Melbourne Fashion Week and has partnered with organizations that support mental health awareness.

With over 2 million subscribers on YouTube and millions of views across her social media platforms, Cassandra Tai is a prominent figure in the beauty and lifestyle influencer space.



2026-01-28 01:45:33,059 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2026-01-28 01:45:44,311 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


[Using RAG]:
According to the provided information, Cassandra Tai is the author of a research paper titled "Applications- GenAI vs. Human Fact-checker" published in 2025.


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
### Pure python retrieval
doc_text = " ".join([doc.text for doc in documents])
corpus = [sent.strip() for sent in doc_text.split('.') if len(sent) > 20]

# Question = "What is difference between prompt sensitivity and model sensitivity?"

# Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus + [question])

# Cosine similarity
similarities = cosine_similarity(X[-1], X[:-1])

# Top 3 sentences
top_k = 3
top_indices = np.argsort(similarities[0])[-top_k:][::-1]

print(f"Question: {question}\n")
for idx in top_indices:
    score = similarities[0][idx]
    print(f"[score {score:.4f}] {corpus[idx]}")

Question: What is difference between prompt sensitivity and model sensitivity?

[score 0.3950] 8/13
Step 3-Stability Evaluation:Reduce cross-prompt/model variance
Prompt Sensitivity
- Test multiple prompt templates / phrasings
- Assess variation in metrics across prompts
Model sensitivity
- Compare models (GPT, Llama, Deepseek, etc
[score 0.0754] Results: GenAI has potential but is fundamentally limited in its
capacity to detect political content credibility
[score 0.0730] )
- Examine where models disagree with humans and with each other 9/13
Step 4-Explainability & Oversight: Automated systems require oversight
Use model-generated rationales as auditable artifacts
Audit for:
- Logical coherence and conceptual validity
- Biases, hallucinations, ethical red flags
Two-way human–AI collaboration
- LLM rationales expand human awareness
- Human experts correct, constrain, and document model behavior 10/13
Step 5-Uncertainty & Error Correction:Acknowledging AI uncertainties
Misclassification

In [ ]:
### Parameter tuning 

question2 = "Summarize the entire document including methodology, results, and conclusion."

# top-k = 1
engine_k1 = index.as_query_engine(similarity_top_k=1)
response_k1 = engine_k1.query(question2)
print(f"[Answer (Top-k=1)]: {response_k1}\n")

# top-k = 5
engine_k5 = index.as_query_engine(similarity_top_k=5)
response_k5 = engine_k5.query(question2)
print(f"[Answer (Top-k=5)]: {response_k5}")

2026-01-28 01:59:37,472 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2026-01-28 01:59:57,629 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


[Answer (Top-k=1)]: The Conception Task Prompts Gold section outlines a framework for creating high-quality labels or clusters as gold standards. This involves two main approaches: zero-shot learning and few-shot learning with fine-tuning.

In the zero-shot approach, machine learning systems can recover gold labels without any prior training data. The focus is on reliability metrics such as Cohen's kappa, Krippendorff's alpha, precision, recall, F1 score, AUC, and Matthews Correlation Coefficient (MCC).

The few-shot approach involves fine-tuning the machine learning system with a small amount of labeled data. In this case, the focus shifts to validity metrics like precision, recall, F1 score, and AUC.

A third approach is to create clusters of materials without prior schema using zero-shot learning. This method relies on reliability metrics such as Silhouette coefficient, pair comparison/adjusted normalized mutual information, etc.

The Like-human subject section discusses a human-cen

2026-01-28 02:00:07,370 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2026-01-28 02:00:25,003 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


[Answer (Top-k=5)]: The importance of acknowledging AI uncertainties in machine learning systems is highlighted. The framework presented emphasizes the need for a systematic approach to validating and documenting LLM use to ensure credible inferences.

The methodology involves designing a dual-track system with reliability and validity tracks. Reliability track metrics include Cohen's kappa, Krippendorff's alpha, and other measures of inter-rater agreement. Validity track metrics encompass precision, recall, F1 score, AUC, MCC, and other measures of accuracy. The silicon participant track focuses on mean/SD relative to human baseline data and human-ness metrics like fluency, cohesiveness, objectivity, readability, etc.

The results demonstrate the effectiveness of this framework in evaluating LLMs' role in various tasks. By acknowledging AI uncertainties and treating labels as noisy measurements, researchers can develop more robust methods for regression and descriptive analyses.

In c

In [13]:
### Vector Embeddings Visualization

query_str = "Social Science"
query_embedding = Settings.embed_model.get_query_embedding(query_str)

print(f"Vector dimension of question '{query_str}': {len(query_embedding)}")
print(f"First 10 elements of the vector: {query_embedding[:10]}")

# These numbers collectively represent the semantic meaning

2026-01-28 01:43:16,178 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"


Vector dimension of question 'Social Science': 4096
First 10 elements of the vector: [-0.016794328, -0.006202411, 0.017395807, 0.0011353239, -0.0016906768, 0.01175532, -0.019479135, 0.017811432, -0.004520285, 0.017569028]
